# C1 - New

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.metrics import accuracy_score, roc_auc_score

## Import Data

In [86]:
# Load datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [87]:
train_data = train_data.dropna()
train_data.reset_index(drop=True, inplace=True)

In [88]:
# Step 1: Handle Missing Values
#def impute_missing_values(data):
#    data['num_sold'] = data.groupby(['country', 'store', 'product'])['num_sold'].transform(
#        lambda x: x.fillna(x.median())
#    )
#    return data

#train_data = impute_missing_values(train_data)

## Handle Date

In [89]:
# Step 2: Feature Engineering
# Extract date-related features
def extract_date_features(data):
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['day'] = data['date'].dt.day
    data['weekday'] = data['date'].dt.weekday
    data['weekofyear'] = data['date'].dt.isocalendar().week.astype(int)
    return data.drop(columns=['date'])

train_data = extract_date_features(train_data)
test_data = extract_date_features(test_data)

# Encode categorical features
encoder = LabelEncoder()
for col in ['country', 'store', 'product']:
    train_data[col] = encoder.fit_transform(train_data[col])
    test_data[col] = encoder.transform(test_data[col])

## Feature Engineering
### Weekend

In [90]:
train_data['is_weekend'] = (train_data['weekday'] == 6) | (train_data['weekday'] == 7)
test_data['is_weekend'] = (test_data['weekday'] == 6) | (test_data['weekday'] == 7)
train_data['is_weekend'] = train_data['is_weekend'].apply(lambda x: 1 if x == 'True' else 0)
test_data['is_weekend'] = test_data['is_weekend'].apply(lambda x: 1 if x == 'True' else 0)

### Year ^ 2

In [91]:
train_data['y2'] = train_data['year'] ** 2
test_data['y2'] = test_data['year'] ** 2

### Group Categorical Features

In [92]:
#train_data['CS'] = train_data['country'] + train_data['store']
#train_data['SP'] = train_data['store'] + train_data['product']
#train_data['CP'] = train_data['country'] + train_data['product']

#test_data['CS'] = test_data['country'] + test_data['store']
#test_data['SP'] = test_data['store'] + test_data['product']
#test_data['CP'] = test_data['country'] + test_data['product']

### Season

In [93]:
train_data['summer'] = (train_data['month'].isin([6, 7, 8])).astype(int)
train_data['autumn'] = (train_data['month'].isin([9, 10, 11])).astype(int)
train_data['winter'] = (train_data['month'].isin([12, 1, 2])).astype(int)
train_data['spring'] = (train_data['month'].isin([3, 4, 5])).astype(int)

test_data['summer'] = (test_data['month'].isin([6, 7, 8])).astype(int)
test_data['autumn'] = (test_data['month'].isin([9, 10, 11])).astype(int)
test_data['winter'] = (test_data['month'].isin([12, 1, 2])).astype(int)
test_data['spring'] = (test_data['month'].isin([3, 4, 5])).astype(int)

In [94]:
train_data.corr()['num_sold']

id           -0.040866
country       0.129113
store         0.239883
product       0.004255
num_sold      1.000000
year         -0.040462
month        -0.006255
day           0.001137
weekday       0.069613
weekofyear   -0.006296
is_weekend         NaN
y2           -0.040486
summer       -0.003762
autumn       -0.015320
winter        0.008759
spring        0.010310
Name: num_sold, dtype: float64

## Train-Test Split

In [95]:
# Step 3: Prepare Data for Modeling
X = train_data.drop(columns=['id', 'num_sold'])
y = train_data['num_sold']
X_test = test_data.drop(columns=['id'])

#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = X[:200000], X[200001:], y[:200000], y[200001:]

## CatBoost Model

In [96]:
model_ctb = CatBoostRegressor(iterations=5000, loss_function='MAPE')

model_ctb.fit(X_train, y_train)   #0.1

0:	learn: 0.8775962	total: 38.8ms	remaining: 3m 13s
1:	learn: 0.8535028	total: 76ms	remaining: 3m 9s
2:	learn: 0.8301939	total: 113ms	remaining: 3m 8s
3:	learn: 0.8076590	total: 150ms	remaining: 3m 7s
4:	learn: 0.7858658	total: 186ms	remaining: 3m 6s
5:	learn: 0.7647964	total: 223ms	remaining: 3m 5s
6:	learn: 0.7444369	total: 269ms	remaining: 3m 11s
7:	learn: 0.7247897	total: 307ms	remaining: 3m 11s
8:	learn: 0.7058049	total: 344ms	remaining: 3m 10s
9:	learn: 0.6874980	total: 381ms	remaining: 3m 10s
10:	learn: 0.6698299	total: 421ms	remaining: 3m 11s
11:	learn: 0.6527735	total: 467ms	remaining: 3m 13s
12:	learn: 0.6363255	total: 510ms	remaining: 3m 15s
13:	learn: 0.6204585	total: 565ms	remaining: 3m 21s
14:	learn: 0.6051736	total: 602ms	remaining: 3m 20s
15:	learn: 0.5904533	total: 639ms	remaining: 3m 18s
16:	learn: 0.5763018	total: 675ms	remaining: 3m 17s
17:	learn: 0.5627095	total: 712ms	remaining: 3m 17s
18:	learn: 0.5496362	total: 757ms	remaining: 3m 18s
19:	learn: 0.5369929	total:

161:	learn: 0.2932187	total: 6.88s	remaining: 3m 25s
162:	learn: 0.2925316	total: 6.93s	remaining: 3m 25s
163:	learn: 0.2923480	total: 6.98s	remaining: 3m 25s
164:	learn: 0.2920658	total: 7.02s	remaining: 3m 25s
165:	learn: 0.2915867	total: 7.07s	remaining: 3m 25s
166:	learn: 0.2912492	total: 7.11s	remaining: 3m 25s
167:	learn: 0.2910069	total: 7.17s	remaining: 3m 26s
168:	learn: 0.2905496	total: 7.22s	remaining: 3m 26s
169:	learn: 0.2902349	total: 7.26s	remaining: 3m 26s
170:	learn: 0.2896477	total: 7.31s	remaining: 3m 26s
171:	learn: 0.2894845	total: 7.36s	remaining: 3m 26s
172:	learn: 0.2891851	total: 7.4s	remaining: 3m 26s
173:	learn: 0.2887938	total: 7.45s	remaining: 3m 26s
174:	learn: 0.2884018	total: 7.51s	remaining: 3m 27s
175:	learn: 0.2880863	total: 7.57s	remaining: 3m 27s
176:	learn: 0.2877902	total: 7.62s	remaining: 3m 27s
177:	learn: 0.2874790	total: 7.67s	remaining: 3m 27s
178:	learn: 0.2869488	total: 7.72s	remaining: 3m 28s
179:	learn: 0.2866891	total: 7.78s	remaining: 3

321:	learn: 0.2419012	total: 14s	remaining: 3m 23s
322:	learn: 0.2418023	total: 14s	remaining: 3m 23s
323:	learn: 0.2403684	total: 14.1s	remaining: 3m 22s
324:	learn: 0.2402289	total: 14.1s	remaining: 3m 22s
325:	learn: 0.2401487	total: 14.1s	remaining: 3m 22s
326:	learn: 0.2400421	total: 14.2s	remaining: 3m 22s
327:	learn: 0.2399356	total: 14.2s	remaining: 3m 22s
328:	learn: 0.2386777	total: 14.2s	remaining: 3m 22s
329:	learn: 0.2384913	total: 14.3s	remaining: 3m 22s
330:	learn: 0.2372859	total: 14.3s	remaining: 3m 21s
331:	learn: 0.2371841	total: 14.3s	remaining: 3m 21s
332:	learn: 0.2370928	total: 14.4s	remaining: 3m 21s
333:	learn: 0.2369770	total: 14.4s	remaining: 3m 21s
334:	learn: 0.2368622	total: 14.5s	remaining: 3m 21s
335:	learn: 0.2367807	total: 14.5s	remaining: 3m 21s
336:	learn: 0.2365734	total: 14.5s	remaining: 3m 21s
337:	learn: 0.2354789	total: 14.6s	remaining: 3m 20s
338:	learn: 0.2353648	total: 14.6s	remaining: 3m 20s
339:	learn: 0.2341542	total: 14.6s	remaining: 3m 2

477:	learn: 0.1998263	total: 20.2s	remaining: 3m 11s
478:	learn: 0.1997501	total: 20.3s	remaining: 3m 11s
479:	learn: 0.1995757	total: 20.3s	remaining: 3m 11s
480:	learn: 0.1995050	total: 20.3s	remaining: 3m 11s
481:	learn: 0.1994090	total: 20.4s	remaining: 3m 11s
482:	learn: 0.1992998	total: 20.4s	remaining: 3m 10s
483:	learn: 0.1992012	total: 20.5s	remaining: 3m 10s
484:	learn: 0.1989624	total: 20.5s	remaining: 3m 10s
485:	learn: 0.1989058	total: 20.5s	remaining: 3m 10s
486:	learn: 0.1988434	total: 20.6s	remaining: 3m 10s
487:	learn: 0.1986250	total: 20.6s	remaining: 3m 10s
488:	learn: 0.1983377	total: 20.6s	remaining: 3m 10s
489:	learn: 0.1981419	total: 20.7s	remaining: 3m 10s
490:	learn: 0.1978868	total: 20.7s	remaining: 3m 10s
491:	learn: 0.1978010	total: 20.8s	remaining: 3m 10s
492:	learn: 0.1977172	total: 20.8s	remaining: 3m 10s
493:	learn: 0.1975637	total: 20.8s	remaining: 3m 10s
494:	learn: 0.1974410	total: 20.9s	remaining: 3m 9s
495:	learn: 0.1973615	total: 20.9s	remaining: 3

635:	learn: 0.1840793	total: 27.1s	remaining: 3m 5s
636:	learn: 0.1840132	total: 27.1s	remaining: 3m 5s
637:	learn: 0.1839482	total: 27.2s	remaining: 3m 5s
638:	learn: 0.1838623	total: 27.2s	remaining: 3m 5s
639:	learn: 0.1838142	total: 27.2s	remaining: 3m 5s
640:	learn: 0.1837678	total: 27.3s	remaining: 3m 5s
641:	learn: 0.1836844	total: 27.3s	remaining: 3m 5s
642:	learn: 0.1836221	total: 27.4s	remaining: 3m 5s
643:	learn: 0.1835748	total: 27.4s	remaining: 3m 5s
644:	learn: 0.1835277	total: 27.4s	remaining: 3m 5s
645:	learn: 0.1834851	total: 27.5s	remaining: 3m 5s
646:	learn: 0.1834273	total: 27.5s	remaining: 3m 5s
647:	learn: 0.1833710	total: 27.5s	remaining: 3m 5s
648:	learn: 0.1833385	total: 27.6s	remaining: 3m 4s
649:	learn: 0.1832549	total: 27.6s	remaining: 3m 4s
650:	learn: 0.1831735	total: 27.7s	remaining: 3m 4s
651:	learn: 0.1831354	total: 27.7s	remaining: 3m 4s
652:	learn: 0.1830930	total: 27.7s	remaining: 3m 4s
653:	learn: 0.1830542	total: 27.8s	remaining: 3m 4s
654:	learn: 

...

4342:	learn: 0.1106615	total: 3m 9s	remaining: 28.6s
4343:	learn: 0.1106136	total: 3m 9s	remaining: 28.6s
4344:	learn: 0.1106121	total: 3m 9s	remaining: 28.5s
4345:	learn: 0.1106108	total: 3m 9s	remaining: 28.5s
4346:	learn: 0.1106093	total: 3m 9s	remaining: 28.4s
4347:	learn: 0.1106049	total: 3m 9s	remaining: 28.4s
4348:	learn: 0.1106024	total: 3m 9s	remaining: 28.3s
4349:	learn: 0.1105978	total: 3m 9s	remaining: 28.3s
4350:	learn: 0.1105936	total: 3m 9s	remaining: 28.3s
4351:	learn: 0.1105913	total: 3m 9s	remaining: 28.2s
4352:	learn: 0.1105834	total: 3m 9s	remaining: 28.2s
4353:	learn: 0.1105823	total: 3m 9s	remaining: 28.1s
4354:	learn: 0.1105781	total: 3m 9s	remaining: 28.1s
4355:	learn: 0.1105689	total: 3m 9s	remaining: 28s
4356:	learn: 0.1105615	total: 3m 9s	remaining: 28s
4357:	learn: 0.1105565	total: 3m 9s	remaining: 27.9s
4358:	learn: 0.1105517	total: 3m 9s	remaining: 27.9s
4359:	learn: 0.1105071	total: 3m 9s	remaining: 27.9s
4360:	learn: 0.1104657	total: 3m 9s	remaining: 27.

4495:	learn: 0.1092224	total: 3m 15s	remaining: 21.9s
4496:	learn: 0.1092201	total: 3m 15s	remaining: 21.8s
4497:	learn: 0.1092191	total: 3m 15s	remaining: 21.8s
4498:	learn: 0.1092183	total: 3m 15s	remaining: 21.7s
4499:	learn: 0.1092159	total: 3m 15s	remaining: 21.7s
4500:	learn: 0.1092149	total: 3m 15s	remaining: 21.6s
4501:	learn: 0.1092131	total: 3m 15s	remaining: 21.6s
4502:	learn: 0.1092119	total: 3m 15s	remaining: 21.6s
4503:	learn: 0.1091963	total: 3m 15s	remaining: 21.5s
4504:	learn: 0.1091955	total: 3m 15s	remaining: 21.5s
4505:	learn: 0.1091881	total: 3m 15s	remaining: 21.4s
4506:	learn: 0.1091851	total: 3m 15s	remaining: 21.4s
4507:	learn: 0.1091818	total: 3m 15s	remaining: 21.3s
4508:	learn: 0.1091810	total: 3m 15s	remaining: 21.3s
4509:	learn: 0.1091799	total: 3m 15s	remaining: 21.2s
4510:	learn: 0.1091789	total: 3m 15s	remaining: 21.2s
4511:	learn: 0.1091781	total: 3m 15s	remaining: 21.2s
4512:	learn: 0.1091757	total: 3m 15s	remaining: 21.1s
4513:	learn: 0.1091750	total

4649:	learn: 0.1083162	total: 3m 22s	remaining: 15.2s
4650:	learn: 0.1083122	total: 3m 22s	remaining: 15.2s
4651:	learn: 0.1083091	total: 3m 22s	remaining: 15.1s
4652:	learn: 0.1083060	total: 3m 22s	remaining: 15.1s
4653:	learn: 0.1083021	total: 3m 22s	remaining: 15s
4654:	learn: 0.1083012	total: 3m 22s	remaining: 15s
4655:	learn: 0.1082938	total: 3m 22s	remaining: 14.9s
4656:	learn: 0.1082888	total: 3m 22s	remaining: 14.9s
4657:	learn: 0.1082839	total: 3m 22s	remaining: 14.9s
4658:	learn: 0.1082791	total: 3m 22s	remaining: 14.8s
4659:	learn: 0.1082744	total: 3m 22s	remaining: 14.8s
4660:	learn: 0.1082695	total: 3m 22s	remaining: 14.7s
4661:	learn: 0.1082648	total: 3m 22s	remaining: 14.7s
4662:	learn: 0.1082601	total: 3m 22s	remaining: 14.6s
4663:	learn: 0.1082549	total: 3m 22s	remaining: 14.6s
4664:	learn: 0.1082522	total: 3m 22s	remaining: 14.6s
4665:	learn: 0.1082471	total: 3m 22s	remaining: 14.5s
4666:	learn: 0.1082424	total: 3m 22s	remaining: 14.5s
4667:	learn: 0.1082406	total: 3m

4805:	learn: 0.1071165	total: 3m 28s	remaining: 8.41s
4806:	learn: 0.1070886	total: 3m 28s	remaining: 8.36s
4807:	learn: 0.1070876	total: 3m 28s	remaining: 8.32s
4808:	learn: 0.1070863	total: 3m 28s	remaining: 8.28s
4809:	learn: 0.1070850	total: 3m 28s	remaining: 8.23s
4810:	learn: 0.1070840	total: 3m 28s	remaining: 8.19s
4811:	learn: 0.1070817	total: 3m 28s	remaining: 8.15s
4812:	learn: 0.1070794	total: 3m 28s	remaining: 8.1s
4813:	learn: 0.1070784	total: 3m 28s	remaining: 8.06s
4814:	learn: 0.1070774	total: 3m 28s	remaining: 8.02s
4815:	learn: 0.1070750	total: 3m 28s	remaining: 7.97s
4816:	learn: 0.1070721	total: 3m 28s	remaining: 7.93s
4817:	learn: 0.1070698	total: 3m 28s	remaining: 7.88s
4818:	learn: 0.1070674	total: 3m 28s	remaining: 7.84s
4819:	learn: 0.1070657	total: 3m 28s	remaining: 7.8s
4820:	learn: 0.1070615	total: 3m 28s	remaining: 7.75s
4821:	learn: 0.1070592	total: 3m 28s	remaining: 7.71s
4822:	learn: 0.1070575	total: 3m 28s	remaining: 7.67s
4823:	learn: 0.1070547	total: 

4961:	learn: 0.1065150	total: 3m 34s	remaining: 1.65s
4962:	learn: 0.1065116	total: 3m 34s	remaining: 1.6s
4963:	learn: 0.1065102	total: 3m 34s	remaining: 1.56s
4964:	learn: 0.1065093	total: 3m 34s	remaining: 1.51s
4965:	learn: 0.1065084	total: 3m 35s	remaining: 1.47s
4966:	learn: 0.1065062	total: 3m 35s	remaining: 1.43s
4967:	learn: 0.1064997	total: 3m 35s	remaining: 1.39s
4968:	learn: 0.1064921	total: 3m 35s	remaining: 1.34s
4969:	learn: 0.1064910	total: 3m 35s	remaining: 1.3s
4970:	learn: 0.1064884	total: 3m 35s	remaining: 1.25s
4971:	learn: 0.1064859	total: 3m 35s	remaining: 1.21s
4972:	learn: 0.1064794	total: 3m 35s	remaining: 1.17s
4973:	learn: 0.1064784	total: 3m 35s	remaining: 1.13s
4974:	learn: 0.1064750	total: 3m 35s	remaining: 1.08s
4975:	learn: 0.1064741	total: 3m 35s	remaining: 1.04s
4976:	learn: 0.1064731	total: 3m 35s	remaining: 996ms
4977:	learn: 0.1064703	total: 3m 35s	remaining: 953ms
4978:	learn: 0.1064692	total: 3m 35s	remaining: 910ms
4979:	learn: 0.1064683	total: 

In [97]:
# Step 5: Evaluate Models
ctb_predictions = model_ctb.predict(X_val)

ctb_mape = mean_absolute_percentage_error(y_val, ctb_predictions)

print(f"CatBoost MAPE: {ctb_mape}")

CatBoost MAPE: 0.07914335841682867


## Predictions

In [68]:
model_ctb = CatBoostRegressor(iterations=5000, loss_function='MAPE')
model_ctb.fit(X, y)

0:	learn: 0.8776210	total: 42.9ms	remaining: 3m 34s
1:	learn: 0.8534064	total: 83.6ms	remaining: 3m 28s
2:	learn: 0.8299837	total: 128ms	remaining: 3m 32s
3:	learn: 0.8073287	total: 169ms	remaining: 3m 30s
4:	learn: 0.7854502	total: 208ms	remaining: 3m 27s
5:	learn: 0.7642843	total: 249ms	remaining: 3m 27s
6:	learn: 0.7438317	total: 296ms	remaining: 3m 31s
7:	learn: 0.7240893	total: 337ms	remaining: 3m 30s
8:	learn: 0.7050048	total: 377ms	remaining: 3m 28s
9:	learn: 0.6865793	total: 417ms	remaining: 3m 27s
10:	learn: 0.6687842	total: 460ms	remaining: 3m 28s
11:	learn: 0.6516114	total: 504ms	remaining: 3m 29s
12:	learn: 0.6350459	total: 552ms	remaining: 3m 31s
13:	learn: 0.6190703	total: 603ms	remaining: 3m 34s
14:	learn: 0.6036488	total: 644ms	remaining: 3m 34s
15:	learn: 0.5888255	total: 691ms	remaining: 3m 35s
16:	learn: 0.5745519	total: 731ms	remaining: 3m 34s
17:	learn: 0.5608325	total: 772ms	remaining: 3m 33s
18:	learn: 0.5476334	total: 812ms	remaining: 3m 32s
19:	learn: 0.5349165

159:	learn: 0.2871957	total: 8.13s	remaining: 4m 5s
160:	learn: 0.2868276	total: 8.18s	remaining: 4m 5s
161:	learn: 0.2865117	total: 8.22s	remaining: 4m 5s
162:	learn: 0.2862977	total: 8.27s	remaining: 4m 5s
163:	learn: 0.2859886	total: 8.31s	remaining: 4m 5s
164:	learn: 0.2856809	total: 8.37s	remaining: 4m 5s
165:	learn: 0.2854293	total: 8.41s	remaining: 4m 4s
166:	learn: 0.2852848	total: 8.45s	remaining: 4m 4s
167:	learn: 0.2849946	total: 8.5s	remaining: 4m 4s
168:	learn: 0.2847282	total: 8.54s	remaining: 4m 4s
169:	learn: 0.2844628	total: 8.58s	remaining: 4m 3s
170:	learn: 0.2842699	total: 8.63s	remaining: 4m 3s
171:	learn: 0.2840071	total: 8.67s	remaining: 4m 3s
172:	learn: 0.2837453	total: 8.71s	remaining: 4m 2s
173:	learn: 0.2835512	total: 8.75s	remaining: 4m 2s
174:	learn: 0.2832535	total: 8.79s	remaining: 4m 2s
175:	learn: 0.2830313	total: 8.84s	remaining: 4m 2s
176:	learn: 0.2827825	total: 8.88s	remaining: 4m 1s
177:	learn: 0.2825132	total: 8.92s	remaining: 4m 1s
178:	learn: 0

317:	learn: 0.2434391	total: 15.1s	remaining: 3m 41s
318:	learn: 0.2419807	total: 15.1s	remaining: 3m 41s
319:	learn: 0.2418730	total: 15.2s	remaining: 3m 41s
320:	learn: 0.2417665	total: 15.2s	remaining: 3m 41s
321:	learn: 0.2404180	total: 15.3s	remaining: 3m 41s
322:	learn: 0.2402699	total: 15.3s	remaining: 3m 41s
323:	learn: 0.2400079	total: 15.4s	remaining: 3m 41s
324:	learn: 0.2399211	total: 15.4s	remaining: 3m 41s
325:	learn: 0.2398299	total: 15.4s	remaining: 3m 41s
326:	learn: 0.2397390	total: 15.5s	remaining: 3m 41s
327:	learn: 0.2396244	total: 15.5s	remaining: 3m 41s
328:	learn: 0.2378009	total: 15.6s	remaining: 3m 41s
329:	learn: 0.2359992	total: 15.6s	remaining: 3m 40s
330:	learn: 0.2359076	total: 15.7s	remaining: 3m 40s
331:	learn: 0.2358263	total: 15.7s	remaining: 3m 40s
332:	learn: 0.2342715	total: 15.7s	remaining: 3m 40s
333:	learn: 0.2341758	total: 15.8s	remaining: 3m 40s
334:	learn: 0.2340868	total: 15.8s	remaining: 3m 40s
335:	learn: 0.2339714	total: 15.9s	remaining: 

476:	learn: 0.1941519	total: 23.3s	remaining: 3m 40s
477:	learn: 0.1939902	total: 23.3s	remaining: 3m 40s
478:	learn: 0.1938959	total: 23.4s	remaining: 3m 40s
479:	learn: 0.1938031	total: 23.4s	remaining: 3m 40s
480:	learn: 0.1936883	total: 23.4s	remaining: 3m 40s
481:	learn: 0.1935435	total: 23.5s	remaining: 3m 40s
482:	learn: 0.1934289	total: 23.5s	remaining: 3m 39s
483:	learn: 0.1933194	total: 23.6s	remaining: 3m 39s
484:	learn: 0.1931313	total: 23.6s	remaining: 3m 39s
485:	learn: 0.1929332	total: 23.6s	remaining: 3m 39s
486:	learn: 0.1928413	total: 23.7s	remaining: 3m 39s
487:	learn: 0.1926968	total: 23.7s	remaining: 3m 39s
488:	learn: 0.1925901	total: 23.8s	remaining: 3m 39s
489:	learn: 0.1925017	total: 23.8s	remaining: 3m 39s
490:	learn: 0.1924186	total: 23.8s	remaining: 3m 39s
491:	learn: 0.1922841	total: 23.9s	remaining: 3m 38s
492:	learn: 0.1921073	total: 23.9s	remaining: 3m 38s
493:	learn: 0.1920267	total: 24s	remaining: 3m 38s
494:	learn: 0.1919482	total: 24s	remaining: 3m 3

635:	learn: 0.1799380	total: 30.1s	remaining: 3m 26s
636:	learn: 0.1798621	total: 30.1s	remaining: 3m 26s
637:	learn: 0.1797884	total: 30.2s	remaining: 3m 26s
638:	learn: 0.1797203	total: 30.2s	remaining: 3m 26s
639:	learn: 0.1796132	total: 30.3s	remaining: 3m 26s
640:	learn: 0.1795728	total: 30.3s	remaining: 3m 26s
641:	learn: 0.1795355	total: 30.4s	remaining: 3m 26s
642:	learn: 0.1793923	total: 30.4s	remaining: 3m 25s
643:	learn: 0.1793244	total: 30.4s	remaining: 3m 25s
644:	learn: 0.1792596	total: 30.5s	remaining: 3m 25s
645:	learn: 0.1791624	total: 30.5s	remaining: 3m 25s
646:	learn: 0.1790964	total: 30.6s	remaining: 3m 25s
647:	learn: 0.1790035	total: 30.6s	remaining: 3m 25s
648:	learn: 0.1789393	total: 30.6s	remaining: 3m 25s
649:	learn: 0.1788764	total: 30.7s	remaining: 3m 25s
650:	learn: 0.1788336	total: 30.7s	remaining: 3m 25s
651:	learn: 0.1787907	total: 30.8s	remaining: 3m 25s
652:	learn: 0.1787491	total: 30.8s	remaining: 3m 25s
653:	learn: 0.1787097	total: 30.9s	remaining: 

...

4339:	learn: 0.1102230	total: 3m 22s	remaining: 30.8s
4340:	learn: 0.1102221	total: 3m 22s	remaining: 30.7s
4341:	learn: 0.1102213	total: 3m 22s	remaining: 30.7s
4342:	learn: 0.1102173	total: 3m 22s	remaining: 30.6s
4343:	learn: 0.1102162	total: 3m 22s	remaining: 30.6s
4344:	learn: 0.1102151	total: 3m 22s	remaining: 30.5s
4345:	learn: 0.1102111	total: 3m 22s	remaining: 30.5s
4346:	learn: 0.1102092	total: 3m 22s	remaining: 30.4s
4347:	learn: 0.1102042	total: 3m 22s	remaining: 30.4s
4348:	learn: 0.1102030	total: 3m 22s	remaining: 30.4s
4349:	learn: 0.1102020	total: 3m 22s	remaining: 30.3s
4350:	learn: 0.1101978	total: 3m 22s	remaining: 30.3s
4351:	learn: 0.1101969	total: 3m 22s	remaining: 30.2s
4352:	learn: 0.1101961	total: 3m 22s	remaining: 30.2s
4353:	learn: 0.1101935	total: 3m 22s	remaining: 30.1s
4354:	learn: 0.1101923	total: 3m 23s	remaining: 30.1s
4355:	learn: 0.1101914	total: 3m 23s	remaining: 30s
4356:	learn: 0.1101905	total: 3m 23s	remaining: 30s
4357:	learn: 0.1101898	total: 3m

4494:	learn: 0.1097753	total: 3m 28s	remaining: 23.5s
4495:	learn: 0.1097745	total: 3m 29s	remaining: 23.4s
4496:	learn: 0.1097738	total: 3m 29s	remaining: 23.4s
4497:	learn: 0.1097709	total: 3m 29s	remaining: 23.3s
4498:	learn: 0.1097680	total: 3m 29s	remaining: 23.3s
4499:	learn: 0.1097651	total: 3m 29s	remaining: 23.3s
4500:	learn: 0.1097638	total: 3m 29s	remaining: 23.2s
4501:	learn: 0.1097629	total: 3m 29s	remaining: 23.2s
4502:	learn: 0.1097618	total: 3m 29s	remaining: 23.1s
4503:	learn: 0.1097610	total: 3m 29s	remaining: 23.1s
4504:	learn: 0.1097583	total: 3m 29s	remaining: 23s
4505:	learn: 0.1097554	total: 3m 29s	remaining: 23s
4506:	learn: 0.1097544	total: 3m 29s	remaining: 22.9s
4507:	learn: 0.1097514	total: 3m 29s	remaining: 22.9s
4508:	learn: 0.1097507	total: 3m 29s	remaining: 22.8s
4509:	learn: 0.1097495	total: 3m 29s	remaining: 22.8s
4510:	learn: 0.1097487	total: 3m 29s	remaining: 22.7s
4511:	learn: 0.1097479	total: 3m 29s	remaining: 22.7s
4512:	learn: 0.1097451	total: 3m

4649:	learn: 0.1092844	total: 3m 36s	remaining: 16.3s
4650:	learn: 0.1092834	total: 3m 36s	remaining: 16.3s
4651:	learn: 0.1092830	total: 3m 36s	remaining: 16.2s
4652:	learn: 0.1092826	total: 3m 36s	remaining: 16.2s
4653:	learn: 0.1092715	total: 3m 36s	remaining: 16.1s
4654:	learn: 0.1092630	total: 3m 37s	remaining: 16.1s
4655:	learn: 0.1092516	total: 3m 37s	remaining: 16s
4656:	learn: 0.1092379	total: 3m 37s	remaining: 16s
4657:	learn: 0.1092304	total: 3m 37s	remaining: 15.9s
4658:	learn: 0.1092300	total: 3m 37s	remaining: 15.9s
4659:	learn: 0.1092225	total: 3m 37s	remaining: 15.8s
4660:	learn: 0.1092211	total: 3m 37s	remaining: 15.8s
4661:	learn: 0.1092146	total: 3m 37s	remaining: 15.8s
4662:	learn: 0.1092044	total: 3m 37s	remaining: 15.7s
4663:	learn: 0.1092018	total: 3m 37s	remaining: 15.7s
4664:	learn: 0.1092013	total: 3m 37s	remaining: 15.6s
4665:	learn: 0.1092008	total: 3m 37s	remaining: 15.6s
4666:	learn: 0.1092000	total: 3m 37s	remaining: 15.5s
4667:	learn: 0.1091996	total: 3m

4802:	learn: 0.1086172	total: 3m 43s	remaining: 9.16s
4803:	learn: 0.1086160	total: 3m 43s	remaining: 9.12s
4804:	learn: 0.1086145	total: 3m 43s	remaining: 9.07s
4805:	learn: 0.1086139	total: 3m 43s	remaining: 9.02s
4806:	learn: 0.1085993	total: 3m 43s	remaining: 8.98s
4807:	learn: 0.1085942	total: 3m 43s	remaining: 8.93s
4808:	learn: 0.1085804	total: 3m 43s	remaining: 8.88s
4809:	learn: 0.1085677	total: 3m 43s	remaining: 8.84s
4810:	learn: 0.1085594	total: 3m 43s	remaining: 8.79s
4811:	learn: 0.1085586	total: 3m 43s	remaining: 8.74s
4812:	learn: 0.1085576	total: 3m 43s	remaining: 8.7s
4813:	learn: 0.1085566	total: 3m 43s	remaining: 8.65s
4814:	learn: 0.1085528	total: 3m 43s	remaining: 8.6s
4815:	learn: 0.1085522	total: 3m 43s	remaining: 8.56s
4816:	learn: 0.1085462	total: 3m 44s	remaining: 8.51s
4817:	learn: 0.1085454	total: 3m 44s	remaining: 8.46s
4818:	learn: 0.1085446	total: 3m 44s	remaining: 8.42s
4819:	learn: 0.1085346	total: 3m 44s	remaining: 8.37s
4820:	learn: 0.1085281	total: 

4955:	learn: 0.1078428	total: 3m 51s	remaining: 2.05s
4956:	learn: 0.1078401	total: 3m 51s	remaining: 2s
4957:	learn: 0.1078389	total: 3m 51s	remaining: 1.96s
4958:	learn: 0.1078380	total: 3m 51s	remaining: 1.91s
4959:	learn: 0.1078369	total: 3m 51s	remaining: 1.86s
4960:	learn: 0.1078361	total: 3m 51s	remaining: 1.82s
4961:	learn: 0.1078280	total: 3m 51s	remaining: 1.77s
4962:	learn: 0.1078270	total: 3m 51s	remaining: 1.73s
4963:	learn: 0.1078260	total: 3m 51s	remaining: 1.68s
4964:	learn: 0.1078248	total: 3m 51s	remaining: 1.63s
4965:	learn: 0.1078236	total: 3m 51s	remaining: 1.58s
4966:	learn: 0.1078227	total: 3m 51s	remaining: 1.54s
4967:	learn: 0.1078201	total: 3m 51s	remaining: 1.49s
4968:	learn: 0.1078189	total: 3m 51s	remaining: 1.45s
4969:	learn: 0.1078181	total: 3m 51s	remaining: 1.4s
4970:	learn: 0.1078171	total: 3m 51s	remaining: 1.35s
4971:	learn: 0.1078164	total: 3m 51s	remaining: 1.3s
4972:	learn: 0.1078062	total: 3m 51s	remaining: 1.26s
4973:	learn: 0.1078055	total: 3m 

In [69]:
ctb_predictions = model_ctb.predict(X_test)

In [70]:
# Create submission
submission = pd.DataFrame({'id': test_data['id'], 'num_sold': ctb_predictions.astype(int)})
submission.to_csv('new1.csv', index=False)
print("Submission file created as 'submission.csv'")

Submission file created as 'submission.csv'
